In [2]:
import pandas as pd
import numpy as np

In [3]:
slu = pd.read_csv('data/slu-Paid_Parking_Occupancy__Last_30_Days_.csv')
# parking rate only contains null
slu = slu.drop(['PaidParkingRate'], axis = 1)

Add availability information

1. Calculate occupancy percentage: The date and time (minute) of the transaction as recorded
2. Aavailability (bool) : True if has available parking spaces. 
3. Available parking spaces (int): Number of availabel parking spaces. 0 if paid spaces are equal or more than the parking space count. A car might leave before the parking expire, thus paird occupancy could be higher than the parking space count. 

In [4]:
slu['OccupancyPercentage'] = slu['PaidOccupancy']/slu['ParkingSpaceCount']
slu['HasAvailability'] = slu.OccupancyPercentage < 1
slu['AvailableSpace'] = np.where(slu['ParkingSpaceCount'] - slu['PaidOccupancy'] > 0, slu['ParkingSpaceCount'] - slu['PaidOccupancy'], 0)

compute latitude and longitude

In [5]:
location = slu.Location.str[7:-1]
slu.Location.str.split(' ',expand=True)
slu[['Latitude','Longitude']]=location.str.split(' ',expand=True)
slu['Latitude'] = pd.to_numeric(slu['Latitude'])
slu['Longitude'] = pd.to_numeric(slu['Longitude'])

Compute datetime information. 

1. Convert a string of time into a datetime object
2. Check wheter a given date is a US holiday
3. Compute day of the week, hour of the day and minute of the day
5. sort by datetime

In [6]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import time
slu['OccupancyDateTime'] = pd.to_datetime(slu['OccupancyDateTime'],infer_datetime_format=True)
cal = calendar()

dr = pd.date_range(start=slu['OccupancyDateTime'].min(), end=slu['OccupancyDateTime'].max())
holidays = cal.holidays(start=dr.min(), end=dr.max())
slu['Holiday'] = slu['OccupancyDateTime'].isin(holidays)

slu['DayOfTheWeek'] = slu['OccupancyDateTime'].dt.day_of_week
slu['Year'] = slu['OccupancyDateTime'].dt.year
slu['Month'] = slu['OccupancyDateTime'].dt.month
slu['Day'] = slu['OccupancyDateTime'].dt.day
slu['Hour'] = slu['OccupancyDateTime'].dt.hour
slu['Minute'] = slu['OccupancyDateTime'].dt.minute
slu['Second'] = slu['OccupancyDateTime'].dt.second
slu['MinuteOfTheDay'] = slu['Hour'] * 60 * 60 + slu['Minute']  * 60

In [7]:
slu = slu.sort_values(by=['OccupancyDateTime'])

In [8]:
# slu.to_csv('data/clean_data.csv', sep=',')

In [6]:
slu.columns

Index(['OccupancyDateTime', 'PaidOccupancy', 'BlockfaceName', 'SideOfStreet',
       'SourceElementKey', 'ParkingTimeLimitCategory', 'ParkingSpaceCount',
       'PaidParkingArea', 'PaidParkingSubArea', 'ParkingCategory', 'Location',
       'OccupancyPercentage', 'HasAvailability', 'AvailableSpace', 'Latitude',
       'Longitude'],
      dtype='object')

In [12]:
loc_df = slu[['Latitude','Longitude', 'BlockfaceName', 'SourceElementKey','SideOfStreet']]
loc_df = loc_df.drop_duplicates(subset=['SourceElementKey'], keep='first')
loc_df = loc_df.set_index('SourceElementKey')

In [30]:
# loc_df.to_csv('data/paystub_location.csv')